In [69]:
import pandas as pd
import msgpack
import numpy as np

In [2]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cell_tidy_data.columns

Index(['AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate',
       'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region',
       'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96',
       'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'FACS_Date', 'Slice',
       'CellClass', 'l1-umap_0', 'l1-umap_1', 'l1-tsne_0', 'l1-tsne_1',
       'MajorType', 'l2-umap_0', 'l2-umap_1', 'l2-tsne_0', 'l2-tsne_1',
       'SubType', 'l3-umap_0', 'l3-umap_1', 'l3-tsne_0', 'l3-tsne_1',
       'L1CellClass', 'class_tsne_0', 'class_tsne_1', 'class_umap_0',
       'class_umap_1', 'Order', 'RegionName', 'MajorRegion', 'SubRegion',
       'DetailRegion', 'PotentialOverlap (MMB)', 'Anterior (CCF coords)',
       'Posterior (CCF coords)', 'MajorRegionColor', 'SubRegionColor',
       'DissectionRegionColor', 'Replicate'],
      dtype='object')

## Coords

In [4]:
use_cols = [
    'l1-umap_0', 'l1-umap_1', 'l1-tsne_0', 'l1-tsne_1', 'l3-umap_0',
    'l3-umap_1', 'l3-tsne_0', 'l3-tsne_1', 'class_tsne_0', 'class_tsne_1',
    'class_umap_0', 'class_umap_1'
]

In [5]:
cell_tidy_data[use_cols].to_hdf('Coords.HDF', key='data')

## Categorical Data

In [6]:
use_cols = [
    'RegionName', 'MajorRegion', 'SubRegion', 'Region', 'Slice', 'CellClass',
    'MajorType', 'SubType', 'Replicate', 'Pos96', 'Plate', 'Col96', 'Row96',
    'Col384', 'Row384', 'FACS_Date'
]

In [7]:
cell_tidy_data[use_cols].to_hdf('Category.HDF', key='data')


## Palette for categorical Data

In [8]:
region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/dissection_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
sub_region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/sub_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
major_region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/major_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()

cell_class_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/cell_class.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
major_type_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/major_type.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
sub_type_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/sub_type.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()

In [9]:
palette = {
    'Region': region_palette,
    'SubRegion': sub_region_palette,
    'MajorRegion': major_region_palette,
    'SubType': sub_type_palette,
    'MajorType': major_type_palette,
    'CellClass': cell_class_palette,
}

In [10]:
def write_msg(obj, path):
    with open(path, 'wb') as f:
        msgpack.pack(obj, f)
        
def read_msg(path):
    with open(path, 'rb') as f:
        obj = msgpack.unpackb(f.read(), raw=False)
    return obj

write_msg(palette, 'Palette.msg')
palette = read_msg('Palette.msg')

## Continuous Data

In [11]:
use_cols = [
    'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate', 'CH_RateAdj', 'FinalReads',
    'InputReads', 'MappedReads', 'BamFilteringRate', 'MappingRate'
]

In [12]:
cell_tidy_data[use_cols].to_hdf('Continuous.HDF', key='data')

## Genes

In [13]:
total_markers = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusterMethylMarker/TotalPairwiseMarker.msg'
)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
def bad_name(name):
    if name.startswith('Gm') or name.endswith('Rik'):
        return False
    elif name[:2] == name[:2].upper():
        return False
    else:
        return True

total_markers = total_markers[total_markers['gene_name'].apply(bad_name)].copy()
marker_counts = total_markers['gene_id'].value_counts()


In [87]:
use_gene = marker_counts[marker_counts > 290]
use_gene.size

48

In [49]:
import xarray as xr
mcds = xr.open_mfdataset(
    '/home/hanliu/project/mouse_rostral_brain/study/Level1-CellClass/ALL_manual/Adata/Select*mcds'
)

mcds = mcds.sel(gene=use_gene.index)
gene_da = mcds['gene_da']
del gene_da.coords['strand_type']
del gene_da.coords['gene_chrom']
del gene_da.coords['gene_start']
del gene_da.coords['gene_end']

gene_da.load()

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  This is separate from the ipykernel package so we can avoid doing imports until
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  from_openmfds=True,


<xarray.DataArray 'gene_da' (cell: 104340, gene: 2218, mc_type: 2)>
array([[[1.05173148, 1.08491898],
        [1.0722611 , 3.28236253],
        [0.88197009, 0.71610103],
        ...,
        [0.99754248, 1.7850414 ],
        [1.13355753, 1.58089273],
        [1.12043904, 1.08206878]],

       [[1.03470126, 0.82921033],
        [0.89876767, 3.87834622],
        [0.8401846 , 0.67370888],
        ...,
        [0.99434967, 1.71258443],
        [1.12350512, 2.1621864 ],
        [0.7672191 , 1.41146594]],

       [[1.1187993 , 1.19232311],
        [0.94911137, 2.92377214],
        [1.06936168, 2.38446341],
        ...,
        [0.75821802, 1.64602621],
        [1.05960533, 1.79535649],
        [0.58293855, 1.16297842]],

       ...,

       [[1.25350341, 2.22102616],
        [1.0879591 , 2.60903002],
        [1.09847558, 2.19168549],
        ...,
        [1.12777934, 1.54845497],
        [0.83712115, 0.79640036],
        [0.73382716, 0.72690188]],

       [[1.16092603, 4.05867241],
        [0.98917632, 1.66794574],
        [1.06471964, 2.16087908],
        ...,
        [0.74241076, 0.90913075],
        [1.05667357, 1.08655254],
        [0.79014789, 1.48345981]],

       [[1.08169307, 1.30917933],
        [1.10854119, 0.98909117],
        [1.07833036, 1.08031449],
        ...,
        [0.62802927, 0.91756429],
        [0.91595007, 0.51985276],
        [1.2488717 , 0.92608237]]])
Coordinates:
  * mc_type  (mc_type) object 'CGN' 'CHN'
  * gene     (gene) object 'ENSMUSG00000035620.15' ... 'ENSMUSG00000026018.12'
  * cell     (cell) object '1A_M_0' '1A_M_1' ... '8J_M_1291' '8J_M_1292'

In [57]:
gene_da.to_netcdf('Gene.mcds')

In [88]:
gene_da.sel(gene=use_gene.index).astype('float32').to_netcdf('Gene.small.mcds')